# [Issue# 1897 Additional Visuals for PUC Analysis](https://github.com/cal-itp/data-analyses/issues/1897)

Received list of transit operators cohorts that may be exempt from efficiency reporting, per PUC 99314.11, .6 and .7. 
- create visuals based on grouping set by list
- recreate visuals based on previous notebook work

## [99314.6](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.6.&lawCode=PUC)
>`funds shall be allocated for operating or capital purpose` pursuant to Sections 99313 and 99314 to an operator `if the operator meets either of the following efficiency standards`:
>- (A) `The operator shall receive its entire allocation`, and any or all of this allocation may be used for operating purposes, if the operator’s `total operating cost per revenue vehicle hour` in the latest year for which audited data are available `does not exceed the sum of the preceding year’s total operating cost per revenue vehicle hour and an amount equal to the product of the percentage change in the Consumer Price Index for the same period multiplied by the preceding year’s total operating cost per revenue vehicle hour.`
>- (B) The operator shall receive its entire allocation, and any or all of this allocation may be used for operating purposes, `if the operator’s average total operating cost per revenue vehicle hour` in the latest three years for which audited data are available `does not exceed the sum of the average of the total operating cost per revenue vehicle hour in the three years preceding the latest year for which audited data are available and an amount equal to the product of the average percentage change in the Consumer Price Index for the same period multiplied by the average total operating cost per revenue vehicle hour in the same three years`.
## [99314.7 (mainly MTC specific)](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PUC&sectionNum=99314.7.)
>the `Metropolitan Transportation Commission` shall apply the following eligibility standards to the operators within the region subject to its jurisdiction:

# [99314.11](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.11.&nodeTreePath=17.11.2.8&lawCode=PUC)
>`Sections 99314.6 and 99314.7 do not apply to an operator for a fiscal year in which the operator expended from local funding an amount for transit operations not less than the amount the operator expended from local funding for transit operations during the 2018–19 fiscal year.` As used in this subdivision, “local funding” means any nonstate grant funds or other revenues generated by, earned by, or distributed to, an operator.

Meaning, if a transit operator spent local funds >= the local funds spent during FY 2018-2019, they are exempt from meeting efficiency standards(?)

In [59]:
import pandas as pd
import altair as alt
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas

@cache
def gcs_pandas():
    return GCSPandas()

# Read in cohort list data

In [ ]:
data = gcs_pandas().read_csv("gs://calitp-analytics-data/data-analyses/ntd/fbr_local_funding_by_cohorts_2019-2024_compiled.csv")

In [25]:
data.columns = data.columns.str.lower()

In [26]:
display(
    data.info(),
    data.head()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ntd_id       1797 non-null   int64 
 1   urban_rural  1797 non-null   object
 2   cohort       1797 non-null   object
 3   metric       1797 non-null   object
 4   year         1797 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 70.3+ KB


None

,ntd_id,urban_rural,cohort,metric,year
0,91018,Rural,Group A,Farebox Recovery Ratio,2019
1,91062,Rural,Group A,Farebox Recovery Ratio,2019
2,91088,Rural,Group A,Farebox Recovery Ratio,2019
3,91043,Rural,Group A,Farebox Recovery Ratio,2019
4,91036,Rural,Group A,Farebox Recovery Ratio,2019


In [54]:
data.value_counts(
    subset=["urban_rural","metric","cohort","year"]
)

urban_rural  metric                          cohort   year
Urban        Farebox Recovery Ratio          Group B  2023    52
                                                      2024    52
                                                      2019    51
                                                      2020    51
             Local Funding % Change vs 2019  Group B  2024    50
                                                              ..
Rural        Farebox Recovery Ratio          Group A  2023    12
                                                      2021    12
                                             Group C  2024    12
                                             Group A  2024    11
                                                      2022    11
Name: count, Length: 72, dtype: int64

# Read in analysis data from prev notebook

In [60]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
ntd_name = "ntd_operator_data_18_23.parquet"
ntd_all_metrics = gcs_pandas().read_parquet(f"{gcs_path}{ntd_name}")

In [62]:
ntd_all_metrics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 0 to 1613
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ntd_id         1524 non-null   object 
 1   source_agency  1614 non-null   object 
 2   agency_status  1614 non-null   object 
 3   reporter_type  1614 non-null   object 
 4   year           1614 non-null   int64  
 5   total_upt      1291 non-null   float64
 6   total_vrh      1291 non-null   float64
 7   total_pmt      1291 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 113.5+ KB
